In [4]:
from os import *
from os.path import *
from itertools import chain
import os
from sklearn import tree
import pandas as pd
from numpy import *
from pandas import *
from sklearn import svm
from sklearn.neighbors import KNeighborsClassifier
import pytz
import matplotlib.pyplot as plt
from sklearn.cross_validation import train_test_split
from sklearn.metrics import *
from sklearn.ensemble import RandomForestClassifier
import itertools

In [5]:
import itertools


def tp(gt, pred):
    return np.sum((gt==1) &(pred==1))/1.0*np.size(gt)

fridge_power = {1:48,
                2:74,
                3:64,
                4:94,
                5:113}

gt_power = {'summer':{},'winter':{}}
pair_df_dict = {'summer':{},'winter':{}}
transients_dict = {'summer':{},'winter':{}}


In [6]:

results_dic = {}
output_path = os.path.expanduser("~/Dropbox/niomtk_data/eco/downsampled")

classifiers_dict = {"SVM": svm.SVC(), "KNN": KNeighborsClassifier(n_neighbors=3),
                    "DT": tree.DecisionTreeClassifier(),"RF":RandomForestClassifier()}

metric_dict = {"Precision":precision_score, "Recall":recall_score,
               "MCC":matthews_corrcoef, "Accuracy":tp}

In [7]:
def nilm_pair_remove_fridge(home, season, folder_path):
    from copy import deepcopy
    season_path = os.path.join(folder_path, season)
    home_path = season_path+"/"+str(home)+".csv"
    df = pd.read_csv(home_path, index_col=0)
    df.index = pd.to_datetime(df.index)

    df = df.between_time("06:00", "22:00").copy()
    train_size = len(df)/2
    df = df.tail(train_size)
    from nilmtk.feature_detectors.steady_states import find_steady_states, find_steady_states_transients
    from nilmtk.disaggregate.hart_85 import Hart85
    h = Hart85()

    fridge_min = fridge_power[home]-30
    fridge_max = fridge_power[home]+30

    # Train on DF to get pairs.
    # 1. Between pairs put occupancy as 1
    # 2. If not a pair, then also put occupancy as 1
    ss, tr = find_steady_states(df[["power"]])
    pred = pd.Series(np.zeros(len(df)),name="occupancy", index=df.index)


    # Find unique days
    days = pd.DatetimeIndex(np.unique(df.index.date))
    for day in days:
        events_day = tr[day.strftime("%Y-%m-%d")].abs()
        # Find first non-fridge event
        event_df = events_day[(events_day['active transition']<=fridge_min)|(events_day['active transition']>=fridge_max)]

        if len(event_df)>0:
            first_event=event_df.index[0]
            last_event = event_df.index[-1]
            pred[day.strftime("%Y-%m-%d")][:first_event]=1
            pred[day.strftime("%Y-%m-%d")][last_event:]=1
    for ix, row in tr.iterrows():

        if not(fridge_min<=row['active transition']<=fridge_max):
            pred.ix[tr.index] = 1
    h.transients = deepcopy(tr)

    pair_df = h.pair(buffer_size=20,
              min_tolerance=100, percent_tolerance=0.035,
              large_transition=1000)
    pair_df_dict[season][home] = pair_df
    transients_dict[season][home] = h.transients
    for idx, row in pair_df.iterrows():
        start = row['T1 Time']
        end = row['T2 Time']
        if not(fridge_min<=row['T1 Active']<=fridge_max):
            time_delta = (end - start)/np.timedelta64(1, 'h')
            if time_delta<24:
                pred[start:end] = 1
            else:
                print "*"*80


    pred_resampled = pred.resample("15T", how="max")
    return pred_resampled

In [10]:
out = {}
accuracy_dict = {}
season = "summer"
folder_path = os.path.expanduser("~/Dropbox/niomtk_data/eco/downsampled")

season_path = os.path.join(folder_path, season)
all_homes = [1,2,3,4,5]
home = 1
    
train_homes = [h for h in all_homes if h!=home]

home_path = season_path+"/"+str(home)+".csv"
df = pd.read_csv(home_path, index_col=0)
df.index = pd.to_datetime(df.index)
df = df.between_time("06:00", "22:00")
train_size = len(df)/2
X_test = df.tail(train_size)
y_test = X_test.pop("occupancy")


gt = y_test.resample("15T", how="max")
        

out = nilm_pair_remove_fridge(home, season,os.path.expanduser("~/Dropbox/niomtk_data/eco/1min/"))

        


Finding Edges, please wait ...
Edge detection complete.
Creating transition frame ...
Transition frame created.
Creating states frame ...
States frame created.
Finished.
********************************************************************************
********************************************************************************
********************************************************************************
********************************************************************************
********************************************************************************
********************************************************************************
********************************************************************************


In [12]:
%matplotlib qt

In [13]:
out.plot()

In [14]:
gt.plot()

In [15]:
df.plot()

In [16]:
 from copy import deepcopy
season_path = os.path.join(folder_path, season)
home_path = season_path+"/"+str(home)+".csv"
df = pd.read_csv(home_path, index_col=0)
df.index = pd.to_datetime(df.index)

df = df.between_time("06:00", "22:00").copy()
train_size = len(df)/2
df = df.tail(train_size)

In [18]:
season_path

'/Users/nipunbatra/Dropbox/niomtk_data/eco/downsampled/summer'

In [21]:
df = pd.read_csv("/Users/nipunbatra/Dropbox/niomtk_data/eco/1min/summer/1.csv", index_col=0)
df = df.tail(len(df)/2)
df.index = pd.to_datetime(df.index)

In [23]:
df.plot()

In [24]:
ax = df.plot(subplots=True)

In [25]:
out.plot(ax=ax[1], color='g')

In [26]:
from common_functions import *

In [28]:
new_df = pd.DataFrame()

In [42]:
latexify(columns=1, fig_height=3)
fig, ax = plt.subplots(nrows=3, sharex=True)
df["2012-08-04 06:00":"2012-08-04 10:00"].power.plot(ax=ax[0])
df["2012-08-04 06:00":"2012-08-04 10:00"].occupancy.plot(ax=ax[1])
out["2012-08-04 06:00":"2012-08-04 10:00"].plot(ax=ax[2])
format_axes(ax[0])
format_axes(ax[1])
format_axes(ax[2])
ax[0].set_ylabel("Power (W)")
ax[0].set_ylim((-100, 2500))
ax[1].set_ylim((-0.1, 1.1))
ax[2].set_ylim((-0.1, 1.1))
ax[0].get_xaxis().set_ticks([])
ax[1].get_xaxis().set_ticks([])
ax[1].get_yaxis().set_ticks([])
ax[2].get_yaxis().set_ticks([])









ax[1].set_ylabel("Occupancy\n Ground truth")
ax[2].set_ylabel("Occupancy\n Predicted")
ax[2].set_xlabel("Time")
plt.tight_layout()
plt.savefig("/Users/nipunbatra/git/NIOMTK/figures/eco_low_accuracy.pdf", bbox_inches="tight")